In [1]:
import pandas as pd
import torch
import torch.nn as nn
from transformer.modules import TransformerModule
from transformer.tokenizer import get_tokenizer
from transformer.dataset import TranslationDataset
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

### Dataset

In [2]:
dataset = pd.read_csv("data/wmt14_translate_de-en_train.csv", lineterminator='\n')

In [3]:
dataset.head()

,de,en
0,An der B 211 befindet sich in Loyermoor der so...,Here the largest town of the district is locat...
1,Ich begrüße die Erklärung des Herrn Kommissar ...,"I should like, in passing, to pay tribute to t..."
2,"Das ist das Gegenteil von dem, was getan werde...",That is the opposite of what should be done an...
3,.,.
4,The Ethnographical museum in Varna is in a hou...,It was designed by the Viennese architect Rupp...


In [4]:
target_lang = "en"
source_lang = "de"
len(dataset)

4508785

In [5]:
dataset["word_count"] = dataset[source_lang].apply(lambda x: len(x.split()))

In [6]:
# min = 1; max = 2937
dataset["word_count"].value_counts(
    ascending=False,
    bins=2937//10
)

(11.02, 21.041]                 1739900
(21.041, 31.061]                1033768
(-1.9369999999999998, 11.02]     969093
(31.061, 41.082]                 455500
(41.082, 51.102]                 186776
                                 ...   
(1534.133, 1544.154]                  0
(1544.154, 1554.174]                  0
(1554.174, 1564.195]                  0
(1564.195, 1574.215]                  0
(1463.99, 1474.01]                    0
Name: count, Length: 293, dtype: int64

In [7]:
dataset["word_count"].apply(
    lambda x: x <= 10
).value_counts()

word_count
False    3723814
True      784971
Name: count, dtype: int64

In [8]:
## Sampling the dataset by Sentence Length and Sorting in Ascending Order
dataset = dataset.loc[
    dataset["word_count"].apply(
        lambda x: x <= 10
    )
].sort_values(
    by="word_count",
    ascending=True
).drop(columns="word_count")

In [9]:
### Dropping duplicates
dataset = dataset.drop_duplicates()

Sentences -> Tokens (Split of Sentence into constituent components) -> Embedding (Every token becomes an embedding)

### Tokenizers

In [10]:
from tokenizers import normalizers, pre_tokenizers, Tokenizer
from tokenizers.pre_tokenizers import Whitespace, Digits, Punctuation
from tokenizers.normalizers import NFD, Lowercase
from tokenizers.trainers import WordLevelTrainer
from tokenizers.models import WordLevel

In [11]:
def get_tokenizer(ds: pd.DataFrame, lang: str, tokenizer_path: str, vocab_size: int = 4096):
    
    assert lang in ds.columns, f"{lang} should be a column in the dataset"
    
    tokenizer= Tokenizer(WordLevel(unk_token="<unk>"))
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
        Whitespace(),
        Digits(individual_digits=True),
        Punctuation()
    ])
    tokenizer.normalizer= normalizers.Sequence([NFD(), Lowercase()])
    trainer = WordLevelTrainer(
        vocab_size= vocab_size,
        min_frequency= 5,
        show_progress= True,
        special_tokens= ["<pad>", "<sos>", "<eos>", "<unk>"]
    )
    tokenizer.train_from_iterator(ds[lang].unique(), trainer= trainer)
    tokenizer.save(tokenizer_path)
    print(f"Tokenizer saved at: {tokenizer_path}")

    return tokenizer


In [12]:
def get_tokenizer_from_path(tokenizer_path: str):
    return Tokenizer.from_file(tokenizer_path)

In [19]:
tokenizer = get_tokenizer(dataset, "en", "./tokenizer/de-en/en_vocab_16192.json", vocab_size=16192)

Tokenizer saved at: ./tokenizer/de-en/en_vocab_16192.json


In [37]:
tokenizer.encode("<sos> neko is cat. 989898 6767 <eos> <pad>").ids

[1, 3, 4826, 3, 4, 26, 51, 26, 51, 26, 51, 52, 50, 52, 50, 2, 0]

In [38]:
tokenizer.decode([1, 3, 9, 3888, 4, 49, 66, 49, 66, 49, 66, 68, 69, 68, 69, 2, 0])

'wir francis . vielen sein vielen sein vielen sein nr er nr er'

### Prepare Dataset using `Dataset`

In [4]:
class TranslationDataset(Dataset):
    def __init__(self,
                ds: pd.DataFrame,
                source_lang: str,
                target_lang: str,
                source_tokenizer_path: str,
                target_tokenizer_path: str,
                max_length: int,
            ):
        self.data = ds
        self.max_length = max_length
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.source_tokenizer= get_tokenizer(source_tokenizer_path)
        self.target_tokenizer= get_tokenizer(target_tokenizer_path)

        self.sos_token = self.source_tokenizer.encode("<sos>").ids
        self.eos_token = self.source_tokenizer.encode("<eos>").ids
        self.pad_token = self.source_tokenizer.encode("<pad>").ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        source_tokens = self.source_tokenizer.encode(row[self.source_lang]).ids[:self.max_length - 2]
        target_tokens = self.target_tokenizer.encode(row[self.target_lang]).ids[:self.max_length - 2]

        source_tokens = self.sos_token + source_tokens + self.eos_token
        target_tokens = self.sos_token + target_tokens
        label_tokens = target_tokens + self.eos_token

        source_tokens += self.pad_token * (self.max_length - len(source_tokens))
        target_tokens += self.pad_token * (self.max_length - len(target_tokens))
        label_tokens += self.pad_token * (self.max_length - len(label_tokens))

        # source tokens, target tokens, label tokens
        return torch.Tensor(source_tokens).to(torch.int64), torch.Tensor(target_tokens).to(torch.int64), torch.Tensor(label_tokens).to(torch.int64)


#### Train Test Split

In [23]:
ds = TranslationDataset(
    dataset,
    "eng",
    "fr",
    "./tokenizer/eng_vocab_4096.json",
    "./tokenizer/fr_vocab_4096.json",
    64
)

In [24]:
train_split = 0.7
test_split = 0.2
val_split = 0.1
train, test, val = random_split(ds, [train_split, test_split, val_split])

In [25]:
train_dataloader = DataLoader(train, batch_size=32)
test_dataloader = DataLoader(test, batch_size=32)
val_dataloader = DataLoader(val, batch_size=32)

### Prepare Custom Function for yielding Batches

In [13]:
class TranslationDataset:
    def __init__(
            self,
            ds: pd.DataFrame,
            source_lang: str,
            target_lang: str,
            source_tokenizer_path: str,
            target_tokenizer_path: str,
            batch_size: int,
            max_length: int,
        ):
        
        self.data = ds
        self.batch_size = batch_size
        self.max_length = max_length
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.source_tokenizer= get_tokenizer_from_path(source_tokenizer_path)
        self.target_tokenizer= get_tokenizer_from_path(target_tokenizer_path)

        self.sos_token = self.source_tokenizer.encode("<sos>").ids
        self.eos_token = self.source_tokenizer.encode("<eos>").ids
        self.pad_token = self.source_tokenizer.encode("<pad>").ids

    def __len__(self):
        return (len(self.data) // self.batch_size) + 1

    def __iter__(self):
        for idx in range(0, len(self.data), self.batch_size):
            batch = self.data.iloc[idx: idx + self.batch_size]
            source_tokens = batch[self.source_lang].apply(lambda x: self.source_tokenizer.encode(x).ids)
            target_tokens = batch[self.target_lang].apply(lambda x: self.target_tokenizer.encode(x).ids)

            source_max_length = source_tokens.apply(len).max()
            target_max_length = target_tokens.apply(len).max()
            max_length = target_max_length if target_max_length > source_max_length else source_max_length
            max_length = self.max_length if max_length > self.max_length else max_length

            source_tokens = source_tokens.apply(lambda x: self.sos_token + x[:max_length - 2] + self.eos_token)
            target_tokens = target_tokens.apply(lambda x: self.sos_token + x[:max_length - 1])
            label_tokens = target_tokens.apply(lambda x: x[:max_length - 1] + self.eos_token)

            source_tokens = source_tokens.apply(lambda x: torch.Tensor(x + self.pad_token * (max_length - len(x))).to(torch.int64))
            target_tokens = target_tokens.apply(lambda x: torch.Tensor(x + self.pad_token * (max_length - len(x))).to(torch.int64))
            label_tokens = label_tokens.apply(lambda x: torch.Tensor(x + self.pad_token * (max_length - len(x))).to(torch.int64))

            yield (
                torch.stack(source_tokens.to_list()),
                torch.stack(target_tokens.to_list()),
                torch.stack(label_tokens.to_list())
            )

In [14]:
train = TranslationDataset(
    ds=dataset,
    source_lang="de",
    target_lang="en",
    source_tokenizer_path="./tokenizer/de-en/de_vocab_16192.json",
    target_tokenizer_path="./tokenizer/de-en/en_vocab_16192.json",
    batch_size=64,
    max_length=128,
)

### Training the Dataset

In [15]:
model = TransformerModule(
    256,
    16192,
    16192,
    128,
    8,
    6,
    0.1,
)

In [27]:
class CustomSchedule(torch.optim.lr_scheduler.LambdaLR):
    def __init__(self, optimizer, d_model, warmup_steps=4000):
        self.d_model = d_model
        self.warmup_steps = warmup_steps

        super(CustomSchedule, self).__init__(optimizer, lr_lambda=self.lr_lambda)

    def lr_lambda(self, step):
        step = max(step, 1)
        lr = (self.d_model ** -0.5) * min(step ** -0.5, step * (self.warmup_steps ** -1.5))
        return lr


target_tokenizer = get_tokenizer_from_path("./tokenizer/de-en/en_vocab_16192.json")
optimizer = torch.optim.Adam(model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9)
scheduler = CustomSchedule(optimizer, 256, 800)
loss = nn.CrossEntropyLoss(ignore_index=target_tokenizer.token_to_id("<pad>"), label_smoothing=0.1)

state = torch.load("./checkpoints/16192-256/de_en_checkpoint-2")
model.load_state_dict(state['model_state_dict'])
optimizer.load_state_dict(state['optimizer_state_dict'])
scheduler.load_state_dict(state["scheduler_state_dict"])

In [22]:
for epoch in range(1, 10):
    model.train()
    batch_iter = tqdm(train, desc= f"Processing Epoch: {epoch:02d}")
    for batch in batch_iter:
        output = model(batch[0], batch[1])
        label = batch[2]

        train_loss = loss(output.view(-1, 16192), label.view(-1))
        batch_iter.set_postfix({"loss": f"{train_loss.item():6.3f}"})

        train_loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad(set_to_none=True)

    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
        }, f"./checkpoints/16192-256/de_en_checkpoint-{epoch}" 
    )


Processing Epoch: 03:  42%|████▏     | 4923/11679 [5:43:40<7:51:38,  4.19s/it, loss=5.774] 


KeyboardInterrupt: 

In [28]:
def inference(model, source_tokenizer, target_tokenizer, input_sentence, max_length=32):
    model.eval()
    input_tokens = source_tokenizer.encode(input_sentence).ids
    input_tensor = torch.tensor([input_tokens], dtype=torch.int64)

    target_tokens = [target_tokenizer.token_to_id("<sos>")]
    target_tensor = torch.tensor([target_tokens], dtype=torch.int64)

    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_tensor, target_tensor)
            next_token = output[0, -1, :].argmax(dim=-1).item()
            target_tokens.append(next_token)
            target_tensor = torch.tensor([target_tokens], dtype=torch.int64)

            # if next_token == target_tokenizer.token_to_id("<eos>"):
            #     break

    print(target_tokens)
    output_sentence = target_tokenizer.decode(target_tokens)
    return output_sentence


In [32]:
test = pd.read_csv("data/wmt14_translate_de-en_test.csv", lineterminator='\n')
test["word_count"] = test[source_lang].apply(lambda x: len(x.split()))
test = test.loc[
    test["word_count"].apply(
        lambda x: x <= 5
    )
].sort_values(
    by="word_count",
    ascending=True
).drop(columns="word_count")
test = test.drop_duplicates()

In [19]:
dataset

,de,en
3,.,.
2625375,Warum?,Why is this?
51295,(Beifall),(Applause)
3566197,.,.
1677975,Beschäftigungsförderung,Community incentive measures in the field of e...
...,...,...
1709050,"Sie bedauert es, daß James Nichols im Gefängni...",It deplores the death of Mr Nichols in prison.
1709043,Sommerschulferien sind in der Tschechischen Re...,Summer school holidays in the Czech Republic a...
1709029,Ich würde auch für die Einschaltung der nation...,I would also counsel in favour of including th...
1709006,"Auch Stirn, Hals, Decolleté, Schultern und Arm...","The forehead, neck, shoulders, sternum and arm..."


In [29]:
for i, r in dataset.iterrows():
    print(inference(
        model,
        get_tokenizer_from_path("./tokenizer/de-en/de_vocab_16192.json"),
        get_tokenizer_from_path("./tokenizer/de-en/en_vocab_16192.json"),
        r['de'],
        128,
    ))

[1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 2, 3, 2, 2, 3, 3, 2, 3, 2, 14, 2, 14, 2, 2, 2, 2, 3, 2, 14, 4, 2, 2, 2, 14, 2, 2, 4, 4, 2, 14, 2, 2, 14, 2, 2, 14, 4, 2, 2, 4, 2, 14, 2, 2, 2, 4, 14, 4, 4, 4, 4, 4, 2, 2, 2, 2, 14, 2, 4, 2, 4, 3, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 14, 2, 14, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 14, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2]
0 0 0 0 . 0 . . 0 0 0 . . 0 . 0 . . . . . 0 . . . . 0 0 . . . . . . 0 . . .
[1, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 2, 2, 2, 14, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 2, 2, 2, 14, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 2, 2, 2, 4, 2, 14, 2, 2, 2, 2, 4, 14, 2, 2, 2, 2, 2, 14, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 14, 2, 3, 4, 14, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14]
. . . . . . . . 0 0 0 0 . 0 . 0 . 0 0 . 0 . 0 0
[1, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2,

KeyboardInterrupt: 